# SQL Alchemy Basics
## Introduction
The code samples provided here showcase some basic functionality within sqlalchemy, including:
* How to create an engine, and establish a connection correctly
* How to run queries on that connection
* Different interpretations and optionss for handling the result

## Requirements
This tutorial just uses an in memory database with no external connections, but if you want to connect to an Oracle db you will need an ODBC driver downloaded. 
I THINK you can find instructions on how to do that here: https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html#installing-cx-oracle-on-windows
 * Python environment able to import the dependencies as listed below
 * (Optional) - a brief look at the tutorial here: https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial

In [ ]:
from sqlalchemy import create_engine, text


## Establishing Connectivity - the Engine

In [ ]:
#In memory connection for purpose of this guide:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

#An actual connection could look more like the following:
#engine = create_engine('oracle+cx_oracle://%s:%s@%s' % (USERNAME, PASSWORD, CON_STRING), echo=True) 


### Important - ensure the connection can close after it has been used
Adjust pool_timeout or use 'with' in python so connection isn't permanent. i.e. use additional argument such as pool_timeout = 58800 in the connection

## Working with Transactions and the DBAPI

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

## Committing Changes

In [ ]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
#I believe commit may have become automatic!

## Adding values to existing table

In [ ]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

Result has lots of methods for fetching and transforming rows, such as the Result.all() method illustrated previously, which returns a list of all Row objects. It also implements the Python iterator interface so that we can iterate over the collection of Row objects directly.

The Row objects themselves are intended to act like Python named tuples. Below we illustrate a variety of ways to access rows.


## Ways to access rows

In [ ]:
#Tuple Assignment - This is the most Python-idiomatic style, which is to assign variables to each row positionally as they are received:
with engine.connect() as conn:

    result = conn.execute(text("select x, y from some_table"))

    for x, y in result:
        ...

In [ ]:
# Integer Index - Tuples are Python sequences, so regular integer access is available too:
#
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))

    for row in result:
        x = row[0]

In [ ]:
# Attribute Name - As these are Python named tuples, the tuples have dynamic attribute names matching the names of each column. These names are normally the names that the SQL statement assigns to the columns in each row. While they are usually fairly predictable and can also be controlled by labels, in less defined cases they may be subject to database-specific behaviors:

with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))

    for row in result:
        y = row.y

        # illustrate use with Python f-strings
        print(f"Row: {row.x} {y}")

In [ ]:
# Mapping Access - To receive rows as Python mapping objects, which is essentially a read-only version of Python’s interface to the common dict object, the Result may be transformed into a MappingResult object using the Result.mappings() modifier; this is a result object that yields dictionary-like RowMapping objects rather than Row objects:
with engine.connect() as conn:

    result = conn.execute(text("select x, y from some_table"))

    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]
        print(x,y)